# Details about the Max-Cut for a random 64-vertex graph

`G64.txt` contains the adjacency matrix of the graph.   
`G64PhiA.txt` contains the output $\Phi_A=\mathrm{Tr}_B(\Phi_{AB})$ of the SDP.

The main difficulty in this example is that the cut achieving the Max-Cut is not unique.  
This results in a mixed output state $\Phi_A$ and hence we need to guess the cuts.

In [1]:
import numpy as np
#import networkx as nx
from itertools import product

In [2]:
W = np.loadtxt("G64.txt")
PhiA = np.loadtxt("G64PhiA.txt")

In [3]:
print("Number of vertices:", W.shape[0])
print("Number of edges:", np.sum(W)/2)

#print("The Adjacency Matrix:\n", W)
#print("The graph6 format:\n", nx.to_graph6_bytes(nx.from_numpy_array(W)))

Number of vertices: 64
Number of edges: 419.0


In [4]:
# The final result of the SDP
# (output of the SCS solver, up to a relative precision ~1e-5)
# This gives an upper bound of the Max-Cut $\xi_2=287$

print("Final result of the SDP:", -np.trace(W@PhiA)/4+np.sum(W)/4)

Final result of the SDP: 286.99854401133337


In [5]:
# Take the first row of PhiA and guess the cut

mu = PhiA[0]
print("The first row of PhiA:\n", mu)

The first row of PhiA:
 [ 1.00001155 -0.02299707 -0.02299569 -0.02299566  0.02299615  0.02299578
  0.02299582 -0.99999008  0.99998912 -0.99998954 -0.02299531 -0.02299545
  0.02299681 -0.0229956   0.02299563 -0.02299574  0.02299608 -0.99998977
  0.62185564 -0.02299573 -0.02299505  0.02299543  0.02299555  0.02299159
 -0.02299933 -0.99998974  0.0229965   0.02299565  0.02299605  0.02299569
 -0.50239185  0.54444631 -0.99998988 -0.02299555  0.02299566  0.02299546
  0.02299604  0.02299535  0.99998986 -0.0229951   0.02299625 -0.99998801
 -0.02299444 -0.02299577 -0.02299562 -0.02299542  0.9999894  -0.02299492
  0.9999999   0.99999134  0.02299596 -0.02299569 -0.02299575  0.9999872
 -0.02299572 -0.99998667 -0.02299591 -0.9999909   0.02299538  0.02299648
  0.99998886 -0.02299555 -0.02299649 -0.02299597]


In [6]:
# Show that $\xi_2=287$ is an achievalbe upper bound
print("The upper bound 287 is achieved by the following cuts:\n")

# number of optimal cuts
num = 0


# initialize the guess
y = np.zeros(64)

# set the entries to one if they are close to
y[mu > 0.9999] = 1
y[mu < -0.9999] = -1

# guess some entries are equal if they are close to each other (one possibility)
y[abs(mu - 0.0229) < 0.0001] = 1
y[abs(mu + 0.0229) < 0.0001] = -1

# run an iteration on the undetermined entries
undet = np.where(np.isclose(y, 0))
for i in product([1, -1], repeat=np.shape(undet)[1]):
    y[undet] = i
    if np.isclose(-y@W@y/4+np.sum(W)/4, 287):
        print("x =", (y).astype(int), "\n")
        #print(np.where(y>0))
        #print("cut_size =", -y@W@y/4+np.sum(W)/4)
        num += 1
    if -y@W@y/4+np.sum(W)/4 > 287.5:
        print("Error!")
        
        
# initialize the guess
z = np.zeros(64)

# set the entries to one if they are close to
z[mu > 0.9999] = 1
z[mu < -0.9999] = -1

# guess some entries are equal if they are close to each other (the other possibility)
z[abs(mu - 0.0229) < 0.0001] = -1
z[abs(mu + 0.0229) < 0.0001] = 1

# run an iteration on the undetermined entries
undet = np.where(np.isclose(z, 0))
for i in product([1, -1], repeat=np.shape(undet)[1]):
    z[undet] = i    
    if np.isclose(-z@W@z/4+np.sum(W)/4, 287):
        print("x =", (z).astype(int), "\n")
        #print(np.where(z>0))
        #print("cut_size =", -z@W@z/4+np.sum(W)/4)
        num += 1
    if -z@W@z/4+np.sum(W)/4 > 287.5:
        print("Error!")
        
print("Number of optimal cuts found:", num)

The upper bound 287 is achieved by the following cuts:

x = [ 1 -1 -1 -1  1  1  1 -1  1 -1 -1 -1  1 -1  1 -1  1 -1  1 -1 -1  1  1  1
 -1 -1  1  1  1  1  1  1 -1 -1  1  1  1  1  1 -1  1 -1 -1 -1 -1 -1  1 -1
  1  1  1 -1 -1  1 -1 -1 -1 -1  1  1  1 -1 -1 -1] 

x = [ 1 -1 -1 -1  1  1  1 -1  1 -1 -1 -1  1 -1  1 -1  1 -1  1 -1 -1  1  1  1
 -1 -1  1  1  1  1 -1  1 -1 -1  1  1  1  1  1 -1  1 -1 -1 -1 -1 -1  1 -1
  1  1  1 -1 -1  1 -1 -1 -1 -1  1  1  1 -1 -1 -1] 

x = [ 1 -1 -1 -1  1  1  1 -1  1 -1 -1 -1  1 -1  1 -1  1 -1 -1 -1 -1  1  1  1
 -1 -1  1  1  1  1  1  1 -1 -1  1  1  1  1  1 -1  1 -1 -1 -1 -1 -1  1 -1
  1  1  1 -1 -1  1 -1 -1 -1 -1  1  1  1 -1 -1 -1] 

x = [ 1  1  1  1 -1 -1 -1 -1  1 -1  1  1 -1  1 -1  1 -1 -1  1  1  1 -1 -1 -1
  1 -1 -1 -1 -1 -1 -1  1 -1  1 -1 -1 -1 -1  1  1 -1 -1  1  1  1  1  1  1
  1  1 -1  1  1  1  1 -1  1 -1 -1 -1  1  1  1  1] 

x = [ 1  1  1  1 -1 -1 -1 -1  1 -1  1  1 -1  1 -1  1 -1 -1  1  1  1 -1 -1 -1
  1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1  1  1 -1 -1  1  